In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch

from IPython.display import clear_output

from growing_ca.core.trainer import CaTrainer
from growing_ca.core.utils_vis import to_rgb

In [ ]:
def visualize_batch(vis0, vis1):
    """Visualize a batch of CA states before and after training steps."""
    print("batch (before/after):")
    plt.figure(figsize=[15, 5])
    for i in range(vis0.shape[0]):
        plt.subplot(2, vis0.shape[0], i + 1)
        plt.imshow(vis0[i])
        plt.axis("off")
    for i in range(vis0.shape[0]):
        plt.subplot(2, vis0.shape[0], i + 1 + vis0.shape[0])
        plt.imshow(vis1[i])
        plt.axis("off")
    plt.show()


def plot_loss(loss_log):
    """Plot the loss history on a log10 scale."""
    plt.figure(figsize=(10, 4))
    plt.title("Loss history (log10)")
    plt.plot(np.log10(loss_log), ".", alpha=0.1)
    plt.show()

In [ ]:
# Training configuration
device = torch.device(
    torch.cuda.current_device() if torch.cuda.is_available() else "cpu"
)

TARGET_EMOJI = 2  # Emoji index to train on
EXPERIMENT_TYPE = "Regenerating"  # Options: "Growing", "Persistent", "Regenerating"
n_epoch = 8000  # Number of training epochs

# Initialize the trainer with configuration
trainer = CaTrainer(
    target_emoji_index=TARGET_EMOJI,
    emoji_path="data/emoji.png",
    model_path=f"models/emoji_{TARGET_EMOJI}.pth",
    experiment_type=EXPERIMENT_TYPE,
    device=device,
    channel_n=16,
    target_padding=16,
    lr=2e-3,
    lr_gamma=0.9999,
    betas=(0.5, 0.5),
    batch_size=8,
    pool_size=1024,
    cell_fire_rate=0.5,
)

print(f"Trainer initialized for emoji {TARGET_EMOJI}")
print(f"Experiment type: {EXPERIMENT_TYPE}")
print(f"Device: {device}")

In [ ]:
# Visualize the target emoji
target_img = trainer.load_emoji(TARGET_EMOJI)
plt.figure(figsize=(4, 4))
plt.title(f"Target Emoji (index {TARGET_EMOJI})")
plt.imshow(to_rgb(target_img))
plt.axis("off")
plt.show()

In [ ]:
# Training loop with visualization
loss_log = []

print(f"Starting training for {n_epoch} epochs...")

for i in range(n_epoch + 1):
    # Get batch
    x0, batch = trainer.get_batch()

    # Train for random number of steps
    steps = np.random.randint(64, 96)
    x, loss = trainer.train_step(x0, trainer.pad_target, steps)

    # Update pool if using pattern pool
    if trainer.use_pattern_pool and batch is not None:
        batch.x[:] = x.detach().cpu().numpy()
        batch.commit()

    # Log loss
    loss_log.append(loss.item())

    # Visualize progress every 100 steps
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Epoch {i}/{n_epoch}, Loss: {loss.item():.6f}")

        # Visualize batch
        vis0, vis1 = trainer.visualize_current_state()
        visualize_batch(vis0, vis1)

        # Plot loss history
        plot_loss(loss_log)

        # Save model
        trainer.save_model()

print("Training complete!")